In [1]:
include("../../src/WaterLily.jl")

using Plots
using Statistics

In [2]:
optPois = "Pois"
optSour = "Easy"
optBCon = "Per"

stepbystep = false

computationID = optPois * "_" * optSour * "_" * optBCon * "_"
print(computationID,"\n")

Pois_Easy_Per_


In [3]:
T=Float32
N = (2^7,3*2^7)
D = length(N)
Ng = N .+ 2
Nd = (Ng..., 2)

offset = -0.3


optSour=="Hard" && (SourceFunc(x,y) = (
    -4 *pi^2 * (1/N[1]^2 + 1/N[2]^2) * sin.(2*pi*(x/N[1].+offset)) .* sin.(2*pi*(y/N[2].+offset)) +
    -16*pi^2 * (1/N[1]^2 + 1/N[2]^2) * sin.(8*pi*(x/N[1].+offset)) .* sin.(8*pi*(y/N[2].+offset))
))

optSour=="Hard" && (ManufacSol(x,y) = (
    sin.(2*pi*(x/N[1].+offset)) .* sin.(2*pi*(y/N[2].+offset)) +
    sin.(8*pi*(x/N[1].+offset)) .* sin.(8*pi*(y/N[2].+offset))/4
))

optSour=="Easy" && (SourceFunc(x,y) = (
    4*pi^2 * (
        cos.(2*pi*(x/N[1])) .* (1 .-cos.(2*pi*(y/N[2])))/N[1]^2 +
        (1 .-cos.(2*pi*(x/N[1]))) .* cos.(2*pi*(y/N[2]))/N[2]^2
    )
))

optSour=="Easy" && (ManufacSol(x,y) = (
    (1 .-cos.(2*pi*(x/N[1]))) .* (1 .-cos.(2*pi*(y/N[2]))) 
))


waveNum = 4*pi
optSour=="Middle" && (SourceFunc(x,y) = (
    waveNum^2 * (
        cos.(waveNum*(x/N[1])) .* (1 .-cos.(waveNum*(y/N[2])))/N[1]^2 +
        (1 .-cos.(waveNum*(x/N[1]))) .* cos.(waveNum*(y/N[2]))/N[2]^2
    )
))

optSour=="Middle" && (ManufacSol(x,y) = (
    (1 .-cos.(waveNum*(x/N[1]))) .* (1 .-cos.(waveNum*(y/N[2]))) 
))

false

In [4]:
solIni = rand(Ng...)*10
IniOrig = copy(solIni)
source = zeros(Ng)
μ₀ = ones(Nd)
#optPois=="MulP" && WaterLily.BC!(μ₀,ntuple(zero, D))

optBCon == "Neu" && WaterLily.BC!(solIni)
optBCon == "Per" && WaterLily.BCPer!(solIni)
# BCPer!(source)
# BCPerVec!(μ₀)

X = transpose(reshape([i-1.5 for i=1:Ng[1] for j=1:Ng[2]],Ng[2],Ng[1]))
Y = transpose(reshape([j-1.5 for i=1:Ng[1] for j=1:Ng[2]],Ng[2],Ng[1]))


source = SourceFunc(X,Y)
realSol = ManufacSol(X,Y)
originalSource = copy(source)

realSol .-= mean(realSol[2:end-1,2:end-1])

130×386 Matrix{Float64}:
 -1.0       -1.0       -1.0       …  -1.0       -1.0       -1.0
 -1.0       -1.0       -1.0          -1.0       -1.0       -1.0
 -1.0       -1.0       -0.999999     -0.999999  -1.0       -1.0
 -1.0       -1.0       -0.999998     -0.999998  -1.0       -1.0
 -1.0       -1.0       -0.999996     -0.999996  -1.0       -1.0
 -0.999999  -0.999999  -0.999993  …  -0.999993  -0.999999  -0.999999
 -0.999999  -0.999999  -0.999989     -0.999989  -0.999999  -0.999999
 -0.999998  -0.999998  -0.999985     -0.999985  -0.999998  -0.999998
 -0.999998  -0.999998  -0.99998      -0.99998   -0.999998  -0.999998
 -0.999997  -0.999997  -0.999974     -0.999974  -0.999997  -0.999997
  ⋮                               ⋱                         ⋮
 -0.999998  -0.999998  -0.99998      -0.99998   -0.999998  -0.999998
 -0.999998  -0.999998  -0.999985     -0.999985  -0.999998  -0.999998
 -0.999999  -0.999999  -0.999989     -0.999989  -0.999999  -0.999999
 -0.999999  -0.999999  -0.999993     -0.9

In [5]:
optPois=="Pois" && (pTest = WaterLily.Poisson(solIni, μ₀, source))
optPois=="MulP" && (pTest = WaterLily.MultiLevelPoisson(solIni, μ₀, source))

false

In [6]:
res = []
stepbystep && for i ∈ 1:200
    partRes = WaterLily.solver!(pTest;log=true,tol=1e-12,itmx=20)
    # pTest.x .-= mean(pTest.x[2:end-1,2:end-1]) 
    pythonplot()
    contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],pTest.x[2:end-1,2:end-1];levels=32)
    savefig("bunchFig/" * computationID * "Calculated_"*string(i, pad=4)*".png")
    # pythonplot()
    # contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],log10.(abs.(realSol[2:end-1,2:end-1]-pTest.x[2:end-1,2:end-1]).+1e-12);levels=32)
    # savefig("bunchFig/" * computationID * "Error_"*string(i, pad=4)*".png")

    push!(res,partRes...)
end

!stepbystep && (res = WaterLily.solver!(pTest;log=true,tol=1e-12,itmx=5e3));

#print(res)

5001-element Vector{Float64}:
    8.162678895173293e6
 1622.3464909320617
   73.60352293777126
   25.648690186813976
    7.820743151641087
    6.396981560517368
    3.449351887990914
    3.8566617383794917
    2.4642915459622436
    3.023966015238822
    ⋮
    1.0327522966890468e-8
    1.0480111784458556e-8
    1.0324758257938222e-8
    1.0477306227020791e-8
    1.032199428910687e-8
    1.0474501420639138e-8
    1.0319231060198288e-8
    1.0471697365112666e-8
    1.0316468571014347e-8

In [7]:
pTest.x .-= mean(pTest.x[2:end-1,2:end-1]) 
realSol .-= mean(realSol[2:end-1,2:end-1]) 

130×386 Matrix{Float64}:
 -1.0       -1.0       -1.0       …  -1.0       -1.0       -1.0
 -1.0       -1.0       -1.0          -1.0       -1.0       -1.0
 -1.0       -1.0       -0.999999     -0.999999  -1.0       -1.0
 -1.0       -1.0       -0.999998     -0.999998  -1.0       -1.0
 -1.0       -1.0       -0.999996     -0.999996  -1.0       -1.0
 -0.999999  -0.999999  -0.999993  …  -0.999993  -0.999999  -0.999999
 -0.999999  -0.999999  -0.999989     -0.999989  -0.999999  -0.999999
 -0.999998  -0.999998  -0.999985     -0.999985  -0.999998  -0.999998
 -0.999998  -0.999998  -0.99998      -0.99998   -0.999998  -0.999998
 -0.999997  -0.999997  -0.999974     -0.999974  -0.999997  -0.999997
  ⋮                               ⋱                         ⋮
 -0.999998  -0.999998  -0.99998      -0.99998   -0.999998  -0.999998
 -0.999998  -0.999998  -0.999985     -0.999985  -0.999998  -0.999998
 -0.999999  -0.999999  -0.999989     -0.999989  -0.999999  -0.999999
 -0.999999  -0.999999  -0.999993     -0.9

In [8]:
pythonplot()
plot(res)
plot!(yscale=:log10, minorgrid=true)
savefig(computationID * "Residual.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/Pois_Easy_Per_Residual.png"

In [9]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],pTest.x[2:end-1,2:end-1];levels=32)
savefig(computationID * "Calculated.png")

sys:1: UserWarning: The following kwargs were not used by contour: 'label'


"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/Pois_Easy_Per_Calculated.png"

In [10]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],log10.(abs.(realSol[2:end-1,2:end-1]-pTest.x[2:end-1,2:end-1]).+1e-12);levels=32)
savefig(computationID * "Error.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/Pois_Easy_Per_Error.png"

In [11]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],realSol[2:end-1,2:end-1];levels=32)
savefig(computationID * "Correct.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/Pois_Easy_Per_Correct.png"

In [12]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],originalSource[2:end-1,2:end-1];levels=32)
savefig(computationID * "Source.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/Pois_Easy_Per_Source.png"

In [13]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],IniOrig[2:end-1,2:end-1];levels=32)
savefig(computationID * "InitialSol.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/Pois_Easy_Per_InitialSol.png"